# 🔍 Step 03: 기본 RAG 시스템 구현

이 노트북에서는 Elasticsearch를 사용하여 기본적인 RAG(Retrieval-Augmented Generation) 시스템을 구현합니다.
RAG는 다음과 같은 단계로 동작합니다:

1. 문서를 벡터로 변환하여 저장 (임베딩)
2. 질문이 들어오면 관련된 문서를 검색
3. 검색된 문서를 참고하여 AI가 답변 생성

## 📚 1단계: 필요한 라이브러리 임포트

각 라이브러리의 역할:
- dotenv: 환경 변수 관리
- elasticsearch: 벡터 데이터베이스 연결
- langchain: RAG 시스템 구현을 위한 프레임워크
- OpenAIEmbeddings: 텍스트를 벡터로 변환
- ElasticsearchStore: Elasticsearch를 벡터 저장소로 사용
- ChatOpenAI: GPT 모델을 이용한 답변 생성
- RetrievalQA: 검색-답변 체인 구현

In [ ]:
# 시스템 환경 변수 관련 라이브러리
import os
from dotenv import load_dotenv

# Elasticsearch 연결을 위한 클라이언트 라이브러리
from elasticsearch import Elasticsearch

# LangChain 관련 라이브러리들
from langchain.embeddings import OpenAIEmbeddings  # 텍스트를 벡터로 변환
from langchain.vectorstores import ElasticsearchStore  # Elasticsearch 벡터 저장소
from langchain.chat_models import ChatOpenAI  # ChatGPT 모델
from langchain.chains import RetrievalQA  # 검색-답변 체인

# .env 파일에서 환경 변수 로드
load_dotenv()

## 🔌 2단계: Elasticsearch 연결

Elasticsearch는 우리의 벡터 데이터베이스 역할을 합니다. 
문서의 벡터를 저장하고 검색하는데 사용됩니다.

연결 시 필요한 정보:
1. 서버 URL
2. 사용자 이름
3. 비밀번호

이 정보들은 보안을 위해 .env 파일에 저장되어 있습니다.

In [ ]:
# Elasticsearch 클라이언트 객체 생성
es = Elasticsearch(
    # .env 파일에서 서버 URL 가져오기
    os.getenv('ELASTICSEARCH_URL'),
    
    # 기본 인증 정보 설정
    basic_auth=(
        os.getenv('ELASTICSEARCH_USERNAME'),  # 사용자 이름
        os.getenv('ELASTICSEARCH_PASSWORD')   # 비밀번호
    ),
    
    # 개발 환경에서는 SSL 인증서 검증 비활성화
    # 실제 운영 환경에서는 적절한 SSL 설정 필요
    verify_certs=False
)

# 서버 연결 상태 확인
if es.ping():  # ping() 메서드로 서버가 응답하는지 테스트
    print("✅ Elasticsearch 연결 성공!")
else:
    print("❌ Elasticsearch 연결 실패!")

## 📝 3단계: 샘플 데이터 준비

RAG 시스템을 테스트하기 위한 샘플 문서들을 준비합니다.
실제 환경에서는 이 부분이 데이터베이스나 파일에서 문서를 불러오는 코드로 대체될 수 있습니다.

여기서는 AI 관련 기초 개념들을 담은 간단한 문장들을 사용합니다.

In [ ]:
# 테스트용 샘플 문서 리스트
# 각 문서는 AI 관련 주요 개념을 설명하는 한 문장으로 구성
documents = [
    "인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력을 컴퓨터로 구현하는 기술입니다.",
    "머신러닝은 AI의 한 분야로, 데이터로부터 패턴을 학습하여 의사결정을 수행합니다.",
    "딥러닝은 머신러닝의 한 종류로, 인공신경망을 사용하여 복잡한 패턴을 학습합니다.",
    "자연어 처리(NLP)는 인간의 언어를 컴퓨터가 이해하고 처리할 수 있도록 하는 AI 기술입니다.",
    "RAG(Retrieval-Augmented Generation)는 대규모 언어 모델의 지식을 외부 데이터로 보강하는 기술입니다."
]

## 🔄 4단계: 벡터 저장소 설정

이 단계에서는 다음과 같은 작업을 수행합니다:

1. OpenAI의 임베딩 모델을 초기화하여 텍스트를 벡터로 변환할 준비
2. Elasticsearch에 벡터를 저장할 인덱스 설정
3. 샘플 문서들을 벡터로 변환하여 저장

이렇게 저장된 벡터들은 나중에 질문과 관련된 문서를 검색할 때 사용됩니다.

In [ ]:
# OpenAI의 임베딩 모델 초기화
# 이 모델이 텍스트를 벡터로 변환하는 역할을 담당
embeddings = OpenAIEmbeddings()

# Elasticsearch 벡터 저장소 설정
index_name = "rag-demo"  # 인덱스 이름 지정
vector_store = ElasticsearchStore(
    es_connection=es,  # Elasticsearch 연결 객체
    index_name=index_name,  # 사용할 인덱스 이름
    embedding=embeddings  # 사용할 임베딩 모델
)

# 준비된 문서들을 벡터로 변환하여 저장
vector_store.add_texts(documents)
print(f"✅ {len(documents)}개의 문서가 성공적으로 저장되었습니다.")

## 🤖 5단계: RAG 시스템 구현

이제 실제 RAG 시스템을 구현합니다. 이 시스템은 다음과 같이 동작합니다:

1. 사용자가 질문을 입력
2. 시스템이 질문과 관련된 문서를 검색
3. ChatGPT가 검색된 문서를 참고하여 답변 생성

이를 위해 LangChain의 RetrievalQA 체인을 사용합니다.

In [ ]:
# ChatGPT 모델 초기화
# temperature=0으로 설정하여 일관된 답변이 나오도록 함
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # 사용할 언어 모델
    chain_type="stuff",  # 검색된 문서를 하나의 컨텍스트로 결합
    retriever=vector_store.as_retriever()  # 벡터 저장소를 검색기로 사용
)

## 💬 6단계: 질문-답변 테스트

마지막으로 구현된 RAG 시스템을 테스트합니다.
시스템은 다음 과정을 거쳐 답변을 생성합니다:

1. 질문을 벡터로 변환
2. 벡터 유사도 검색으로 관련 문서 찾기
3. 찾은 문서를 기반으로 ChatGPT가 답변 생성

In [ ]:
# 테스트할 질문 설정
question = "RAG 기술이 무엇인가요?"

# RAG 체인을 통해 답변 생성
# run() 메서드는 내부적으로 다음 과정을 수행:
# 1. 질문을 벡터화
# 2. 유사한 문서 검색
# 3. ChatGPT로 답변 생성
answer = qa_chain.run(question)

# 결과 출력
print(f"질문: {question}")
print(f"답변: {answer}")

# 다른 질문들도 시도해 보세요!
# 예: "머신러닝과 딥러닝의 차이는 무엇인가요?"
# 예: "자연어 처리는 어떤 기술인가요?"